**Refs**
https://github.com/bikz05/ipython-notebooks/blob/master/computer-vision/displaying-video-in-ipython-notebook.ipynb

**PyImageSearch Transfer Learning Tutorial**
    https://www.pyimagesearch.com/2019/05/20/transfer-learning-with-keras-and-deep-learning/

In [1]:
%qtconsole

In [21]:
# one-time prep work
import numpy as np
import matplotlib.pyplot as plt
# set matplotlib to render in this window, with the specified figure size
%matplotlib inline
import pylab
#pylab.rcParams['figure.figsize'] = (12.0, 8.0)
import cv2


Populating the interactive namespace from numpy and matplotlib


In [12]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from imutils import paths


from IPython.display import clear_output
%pylab inline 
pylab.rcParams['figure.figsize'] = (12.0, 8.0)

import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import scipy.ndimage as sciimg
import os

Populating the interactive namespace from numpy and matplotlib


In [55]:
#Video/Data Paths

nonePath = 'Videos/Data/None/None_true.mp4'
itemPath = 'Videos/Data/Tear/Tear_Ww.mp4'

pixstep = 200

In [4]:
def drawGridLines(frame, line_color=(0, 255, 0), thickness=1, type_=cv2.LINE_AA, pxstep=100):
    x = pxstep
    y = pxstep
    while x < frame.shape[1]:
        cv2.line(frame, (x, 0), (x, frame.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        x += pxstep
    while y < frame.shape[0]:
        cv2.line(frame, (0, y), (frame.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
        y += pxstep
        

In [39]:
def drawBox(frame, xpts = (0, 1), ypts = (0, 1), line_color=(255, 0, 0), thickness = 2, type_=cv2.LINE_AA, pxstep=200):
    x1, x2 = xpts
    y1, y2 = ypts
    
    x1*=pxstep
    x2*=pxstep
    y1*=pxstep
    y2*=pxstep
    
    cv2.rectangle(frame, (x1, y1), (x2, y2), line_color, thickness)

In [85]:
#drawBox(frame, (1, 2), (3, 4),  pxstep=200)
#pxstep = 200
#frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
print(frame.shape)
def cropFrame(frame, xpts=(x1, x2), ypts=(y1, y2), pxstep=200):
    plt.imshow(frame[3*pxstep:4*pxstep, 1*pxstep:2*pxstep])

(1080, 1920, 3)


In [79]:
def videoToFrameLabel(vidpath, outDir='train_images/', cropped = False, xpts = (1, 2), ypts = (3, 4), pxstep=200):
    #Read Video Frame By Frame
    vid = cv2.VideoCapture(vidpath)
    top = y1*pxstep
    bot = y2*pxstep
    left = x1*pxstep
    right = x2*pxstep
    #plt.imshow(game[top:bot, left:right])

    # Put the code in try-except statements
    # Catch the keyboard exception and 
    # release the camera device and 
    # continue with the rest of code.
    try:
        frame_no = 0
        prefix = outDir + vidpath.split('/')[-1].split('.')[0]
        print("[INFO] Writing to "+outDir+" ...")
        while(True):
            frame_no+=1 
            # Capture frame-by-frame
            ret, frame = vid.read()
            if not ret:
                # Release the Video Device if ret is false
                vid.release()
                # Message to be displayed after releasing the device
                print("Released Video Resource")
                break
            # Convert the image from OpenCV BGR format to matplotlib RGB format
            # to display the image
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            outPath = prefix + '-' + str(frame_no) +'.png'
            if cropped == True: 
                cv2.imwrite(outPath, frame[top:bot, left:right])
            else: 
                cv2.imwrite(outPath, frame)

        print("[INFO] Done writing")
    except KeyboardInterrupt:
        # Release the Video Device
        vid.release()
        # Message to be displayed after releasing the device
        print("[Detected Interupt]: Stopped writing at frame " + str(frame) + ", released video resource")


In [90]:
itemPath = 'Videos/Data/Tear/Tear_enemy.mp4'
#itemPath = 'Videos/Data/None/None_Varus.mp4'
#videoToFrameLabel(itemPath, outDir='train_images/none/', cropped = True)
videoToFrameLabel(itemPath, outDir='validation_images/tear/', cropped = False)

#videoToFrameLabel(itemPath, outDir='train_images/tear/')

[INFO] Writing to validation_images/tear/ ...
Released Video Resource
[INFO] Done writing


In [65]:
#Read Video Frame By Frame
vidpath = itemPath
# You can also give path to the video file
#itemPath = 'Videos/Data/Tear/Tear_Kha.mp4'
itemPath = 'Videos/Data/None/None_belt_shiv.mp4'
vid = cv2.VideoCapture(vidpath)
x1, x2, y1, y2 = (1, 2, 3, 4)
pxstep=200
top = y1*pxstep
bot = y2*pxstep
left = x1*pxstep
right = x2*pxstep

# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    frame_no = 0
    prefix = vidpath.split('/')[-1].split('.')[0]
    while(True):
        frame_no+=1 
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        drawGridLines(frame, pxstep=200)
        # Turn off the axis
        # Title of the window
        title("Input Stream Frame_No: "+str(frame_no))
        # Display the frame
        imshow(frame[top:bot, left:right])
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource
